<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/212_Customer_Journey_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Node 2: Ticket Creation

### What it does
1. Takes the customer message from state
2. Derives an initial issue type using pattern matching
3. Creates a ticket data structure
4. Adds `ticket_data` to the state

### Files created
1. `utils/ticket_utils.py` — utilities for ticket creation:
   - `derive_issue_type_from_message()` — classifies issue type from message text
   - `create_ticket_data()` — creates the ticket structure

2. `nodes/ticket_creation_node.py` — Node 2 that creates tickets

3. `tests/test_node2_ticket_creation.py` — tests for Node 2

### How it works

**Pattern matching rules:**
- "lost" / "tracking unavailable" → `lost_package`
- "delivered but" / "not received" → `item_not_received`
- "warehouse" → `warehouse_delay`
- "delayed" / "late" → `delivery_delay`
- "where is" / "status" / "tracking" → `where_is_my_order`

**Ticket structure created:**
```python
{
    "ticket_id": "TICKET-C001-O1001-1234567890",
    "customer_id": "C001",
    "order_id": "O1001",
    "customer_message": "...",
    "issue_type": "where_is_my_order",
    "created_at": "2025-11-18T...",
    "status": "open",
    "priority": "medium"
}
```


# ticket_utils.py

In [ ]:
"""Utilities for ticket creation and issue type classification"""

from typing import Dict, Any, Optional
from datetime import datetime
import re


def derive_issue_type_from_message(customer_message: str) -> str:
    """
    Derive initial issue type from customer message using pattern matching.

    This is a simple rule-based classifier. In production, you might use:
    - LLM-based classification
    - ML model
    - More sophisticated NLP

    Args:
        customer_message: The customer's message/inquiry

    Returns:
        Initial issue type string
    """
    message_lower = customer_message.lower()

    # Pattern matching rules (order matters - more specific first)

    # Lost package indicators
    if any(phrase in message_lower for phrase in [
        "lost", "package lost", "is my package lost", "tracking unavailable",
        "tracking says unavailable", "can't find", "disappeared"
    ]):
        return "lost_package"

    # Item not received (delivered but missing)
    if any(phrase in message_lower for phrase in [
        "delivered but", "marked delivered", "says delivered but",
        "nothing at my", "not at my house", "not received",
        "didn't receive", "haven't received"
    ]):
        return "item_not_received"

    # Warehouse delay indicators
    if any(phrase in message_lower for phrase in [
        "warehouse", "hasn't left the warehouse", "still at warehouse",
        "warehouse issue", "warehouse delay"
    ]):
        return "warehouse_delay"

    # Delivery delay indicators
    if any(phrase in message_lower for phrase in [
        "delayed", "delay", "late", "taking too long", "should have arrived",
        "expected delivery", "past due", "overdue"
    ]):
        return "delivery_delay"

    # Status check (where is my order)
    if any(phrase in message_lower for phrase in [
        "where is", "where's", "status", "tracking", "update",
        "hasn't arrived", "when will", "when is it coming",
        "check", "can you check"
    ]):
        return "where_is_my_order"

    # Default fallback
    return "unknown_issue"


def create_ticket_data(
    customer_id: str,
    order_id: str,
    customer_message: str,
    issue_type: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create ticket data structure from customer message.

    Args:
        customer_id: Customer ID
        order_id: Order ID
        customer_message: Customer's message
        issue_type: Optional pre-determined issue type. If not provided,
                   will be derived from message.

    Returns:
        Ticket data dictionary
    """
    # Derive issue type if not provided
    if issue_type is None:
        issue_type = derive_issue_type_from_message(customer_message)

    ticket_data = {
        "ticket_id": f"TICKET-{customer_id}-{order_id}-{int(datetime.now().timestamp())}",
        "customer_id": customer_id,
        "order_id": order_id,
        "customer_message": customer_message,
        "issue_type": issue_type,
        "created_at": datetime.now().isoformat(),
        "status": "open",
        "priority": "medium"  # Will be adjusted by later nodes based on context
    }

    return ticket_data



# ticket creation node


In [ ]:
"""Node 2: Ticket Creation - Derive ticket information from customer message"""

from typing import Dict, Any
from datetime import datetime
from config import CustomerJourneyOrchestratorState, CustomerJourneyOrchestratorConfig
from utils.ticket_utils import create_ticket_data


def ticket_creation_node(
    state: CustomerJourneyOrchestratorState,
    config: CustomerJourneyOrchestratorConfig
) -> Dict[str, Any]:
    """
    Node 2: Ticket Creation

    This node:
    1. Takes the customer message
    2. Derives initial issue type from message
    3. Creates ticket data structure
    4. Adds ticket_data to state

    Args:
        state: Current orchestrator state (should have customer_message from Node 1)
        config: Orchestrator configuration

    Returns:
        Updated state with ticket_data
    """
    errors = state.get("errors", [])
    customer_id = state.get("customer_id")
    order_id = state.get("order_id")
    customer_message = state.get("customer_message")

    # Validate required inputs
    if not customer_id:
        errors.append("customer_id is required for ticket creation")
        return {"errors": errors}

    if not order_id:
        errors.append("order_id is required for ticket creation")
        return {"errors": errors}

    if not customer_message:
        errors.append("customer_message is required for ticket creation")
        return {"errors": errors}

    try:
        # Create ticket data from customer message
        ticket_data = create_ticket_data(
            customer_id=customer_id,
            order_id=order_id,
            customer_message=customer_message
        )

        # Prepare updated state
        updates: Dict[str, Any] = {
            "ticket_data": ticket_data,
            "errors": errors,  # Always include errors field
        }

        return updates

    except Exception as e:
        errors.append(f"Error during ticket creation: {str(e)}")
        return {"errors": errors}



# smoke test

In [ ]:
"""Test Node 2: Ticket Creation"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))

from config import CustomerJourneyOrchestratorState, CustomerJourneyOrchestratorConfig
from nodes.data_aggregation_node import data_aggregation_node
from nodes.ticket_creation_node import ticket_creation_node


def test_ticket_creation_node():
    """Test that Node 2 correctly creates ticket data from customer message"""

    print("\n" + "="*60)
    print("TEST 1: Ticket Creation with Valid Data")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    # First run Node 1 to get data
    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C001",
        "order_id": "O1001",
        "customer_message": "Hi, my order hasn't arrived yet. The tracking hasn't updated in a while. Can you check?",
        "errors": []
    }

    # Run Node 1
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    # Now run Node 2
    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    # Check results
    assert "errors" in node2_result, "Result should contain errors field"

    if node2_result.get("errors"):
        print(f"❌ ERRORS FOUND: {node2_result['errors']}")
        return False

    # Verify ticket_data was created
    assert "ticket_data" in node2_result, "Should have ticket_data"

    ticket = node2_result["ticket_data"]

    # Verify ticket structure
    assert "ticket_id" in ticket, "Should have ticket_id"
    assert "customer_id" in ticket, "Should have customer_id"
    assert "order_id" in ticket, "Should have order_id"
    assert "customer_message" in ticket, "Should have customer_message"
    assert "issue_type" in ticket, "Should have issue_type"
    assert "created_at" in ticket, "Should have created_at"
    assert "status" in ticket, "Should have status"

    # Verify values
    assert ticket["customer_id"] == "C001", "Customer ID should match"
    assert ticket["order_id"] == "O1001", "Order ID should match"
    assert ticket["issue_type"] == "where_is_my_order", "Issue type should be derived correctly"
    assert ticket["status"] == "open", "Status should be open"

    print("✅ All assertions passed!")
    print(f"\nTicket ID: {ticket['ticket_id']}")
    print(f"Issue Type: {ticket['issue_type']}")
    print(f"Status: {ticket['status']}")
    print(f"Created At: {ticket['created_at']}")
    print(f"\nCustomer Message: {ticket['customer_message'][:80]}...")

    return True


def test_issue_type_classification():
    """Test issue type classification for different message types"""

    print("\n" + "="*60)
    print("TEST 2: Issue Type Classification")
    print("="*60)

    from utils.ticket_utils import derive_issue_type_from_message

    test_cases = [
        ("My package is delayed again", "delivery_delay"),
        ("The tracking page says unavailable. Is my package lost?", "lost_package"),
        ("Tracking says delivered but there's nothing at my house", "item_not_received"),
        ("My order hasn't left the warehouse yet", "warehouse_delay"),
        ("Where is my order?", "where_is_my_order"),
        ("Hi, just checking in — any update on the shipping?", "where_is_my_order"),
    ]

    all_passed = True
    for message, expected_type in test_cases:
        actual_type = derive_issue_type_from_message(message)
        passed = actual_type == expected_type
        status = "✅" if passed else "❌"
        print(f"{status} '{message[:50]}...' → {actual_type} (expected: {expected_type})")
        if not passed:
            all_passed = False

    return all_passed


def test_ticket_creation_missing_message():
    """Test error handling for missing customer message"""

    print("\n" + "="*60)
    print("TEST 3: Missing Customer Message")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C001",
        "order_id": "O1001",
        # Missing customer_message
        "errors": []
    }

    result = ticket_creation_node(state, config)

    assert "errors" in result, "Should have errors field"
    assert len(result["errors"]) > 0, "Should have at least one error"
    assert "customer_message" in result["errors"][0].lower(), "Error should mention customer_message"

    print("✅ Error handling works correctly!")
    print(f"Error message: {result['errors'][0]}")

    return True


def test_end_to_end_nodes_1_and_2():
    """Test that Node 1 and Node 2 work together"""

    print("\n" + "="*60)
    print("TEST 4: End-to-End (Node 1 → Node 2)")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    # Initial state
    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C002",
        "order_id": "O1002",
        "customer_message": "My package is delayed again. This is really frustrating. What's going on?",
        "errors": []
    }

    # Run Node 1
    print("Running Node 1: Data Aggregation...")
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    assert "customer_data" in state, "Node 1 should add customer_data"
    assert "order_data" in state, "Node 1 should add order_data"
    assert "logistics_data" in state, "Node 1 should add logistics_data"

    # Run Node 2
    print("Running Node 2: Ticket Creation...")
    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    assert "ticket_data" in state, "Node 2 should add ticket_data"
    assert state["ticket_data"]["issue_type"] == "delivery_delay", "Should classify as delivery_delay"

    print("✅ Both nodes work together correctly!")
    print(f"\nFinal State Keys: {list(state.keys())}")
    print(f"Customer: {state['customer_data']['name']}")
    print(f"Order: {state['order_data']['order_id']}")
    print(f"Ticket Issue Type: {state['ticket_data']['issue_type']}")
    print(f"Logistics Status: {state['logistics_data']['status']}")

    return True


if __name__ == "__main__":
    print("\n🧪 Testing Node 2: Ticket Creation")
    print("="*60)

    results = []

    try:
        results.append(("Test 1: Ticket Creation", test_ticket_creation_node()))
    except Exception as e:
        print(f"❌ Test 1 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 1: Ticket Creation", False))

    try:
        results.append(("Test 2: Issue Classification", test_issue_type_classification()))
    except Exception as e:
        print(f"❌ Test 2 failed with exception: {e}")
        results.append(("Test 2: Issue Classification", False))

    try:
        results.append(("Test 3: Missing Message", test_ticket_creation_missing_message()))
    except Exception as e:
        print(f"❌ Test 3 failed with exception: {e}")
        results.append(("Test 3: Missing Message", False))

    try:
        results.append(("Test 4: End-to-End", test_end_to_end_nodes_1_and_2()))
    except Exception as e:
        print(f"❌ Test 4 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 4: End-to-End", False))

    # Summary
    print("\n" + "="*60)
    print("TEST SUMMARY")
    print("="*60)
    for test_name, passed in results:
        status = "✅ PASSED" if passed else "❌ FAILED"
        print(f"{status}: {test_name}")

    all_passed = all(result[1] for result in results)
    if all_passed:
        print("\n🎉 All tests passed! Node 2 is working correctly.")
    else:
        print("\n⚠️  Some tests failed. Please review the errors above.")

    sys.exit(0 if all_passed else 1)



In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_027_Customer_Journey_Orchestrator % python3 tests/test_node2_ticket_creation.py

🧪 Testing Node 2: Ticket Creation
============================================================

============================================================
TEST 1: Ticket Creation with Valid Data
============================================================
✅ All assertions passed!

Ticket ID: TICKET-C001-O1001-1763500417
Issue Type: where_is_my_order
Status: open
Created At: 2025-11-18T16:13:37.139149

Customer Message: Hi, my order hasn't arrived yet. The tracking hasn't updated in a while. Can you...

============================================================
TEST 2: Issue Type Classification
============================================================
✅ 'My package is delayed again...' → delivery_delay (expected: delivery_delay)
✅ 'The tracking page says unavailable. Is my package ...' → lost_package (expected: lost_package)
✅ 'Tracking says delivered but there's nothing at my ...' → item_not_received (expected: item_not_received)
✅ 'My order hasn't left the warehouse yet...' → warehouse_delay (expected: warehouse_delay)
✅ 'Where is my order?...' → where_is_my_order (expected: where_is_my_order)
✅ 'Hi, just checking in — any update on the shipping?...' → where_is_my_order (expected: where_is_my_order)

============================================================
TEST 3: Missing Customer Message
============================================================
✅ Error handling works correctly!
Error message: customer_message is required for ticket creation

============================================================
TEST 4: End-to-End (Node 1 → Node 2)
============================================================
Running Node 1: Data Aggregation...
Running Node 2: Ticket Creation...
✅ Both nodes work together correctly!

Final State Keys: ['customer_id', 'order_id', 'customer_message', 'errors', 'customer_data', 'order_data', 'logistics_data', 'data_loaded_at', 'ticket_data']
Customer: Mark Johnson
Order: O1002
Ticket Issue Type: delivery_delay
Logistics Status: delayed

============================================================
TEST SUMMARY
============================================================
✅ PASSED: Test 1: Ticket Creation
✅ PASSED: Test 2: Issue Classification
✅ PASSED: Test 3: Missing Message
✅ PASSED: Test 4: End-to-End

🎉 All tests passed! Node 2 is working correctly.


All tests passed. Node 2 is working correctly.

## Test results summary

### Test 1: Ticket Creation — PASSED
- Ticket created with correct structure
- Issue type correctly derived: `where_is_my_order`
- All required fields present

### Test 2: Issue Type Classification — PASSED
All 6 classification tests passed:
- "delayed" → `delivery_delay`
- "lost" / "unavailable" → `lost_package`
- "delivered but" → `item_not_received`
- "warehouse" → `warehouse_delay`
- "where is" / "status" → `where_is_my_order`

### Test 3: Error Handling — PASSED
- Correctly handles missing customer message
- Provides clear error messages

### Test 4: End-to-End (Node 1 → Node 2) — PASSED
This shows the state building up:
```
Final State Keys: [
  'customer_id',
  'order_id',
  'customer_message',
  'errors',
  'customer_data',        ← From Node 1
  'order_data',           ← From Node 1
  'logistics_data',       ← From Node 1
  'data_loaded_at',        ← From Node 1
  'ticket_data'           ← From Node 2 ✨
]
```

## What this means

1. Node 2 works correctly — creates tickets and classifies issue types
2. Nodes work together — Node 1 → Node 2 flow is solid
3. State is building up — the backpack is being filled as expected
4. Pattern matching works — issue types are correctly identified

## Progress so far

- Node 1: Data Aggregation — loads and merges data from multiple sources
- Node 2: Ticket Creation — derives ticket info from customer message

Next up:
- Node 3: Issue Classification — uses decision rules to classify the final issue type (combining ticket + logistics + customer data)
- Node 4: Resolution Path — determines which specialist agents to call
- Node 5: Agent Execution — calls specialist agents
- Node 6: Response Generation — creates final customer response



# decision rules

In [ ]:
"""Decision rules for issue classification and resolution path determination"""

from typing import Dict, Any, List


def classify_issue(
    order: Dict[str, Any],
    ticket: Dict[str, Any],
    customer: Dict[str, Any],
    logistics: Dict[str, Any]
) -> str:
    """
    Classify the final issue type based on order, ticket, customer, and logistics data.

    This implements the decision rules from orchestrator_decision_rules.json.
    The rules check multiple conditions to determine the most appropriate issue type.

    Args:
        order: Order data dictionary
        ticket: Ticket data dictionary (from Node 2)
        customer: Customer data dictionary (merged customer + marketing)
        logistics: Logistics data dictionary

    Returns:
        Final classified issue type string
    """
    # Lost package (highest priority - check logistics status first)
    if logistics.get("status") == "lost":
        return "lost_package"

    # Delivery delay (check churn risk to determine severity)
    if logistics.get("status") == "delayed":
        churn_risk = customer.get("churn_risk", 0)
        if churn_risk > 0.25:
            return "delivery_delay_with_churn_risk"
        return "delivery_delay"

    # Warehouse delay (check churn risk to determine severity)
    if order.get("warehouse_issue_flag") is True:
        churn_risk = customer.get("churn_risk", 0)
        if churn_risk > 0.30:
            return "warehouse_delay_high_churn_risk"
        return "warehouse_delay"

    # Item not received (delivered but missing)
    if logistics.get("status") == "delivered" and ticket.get("issue_type") == "item_not_received":
        churn_risk = customer.get("churn_risk", 0)
        if churn_risk > 0.20:
            return "repeat_item_not_received"
        return "item_not_received"

    # Status checks (friendly vs simple based on engagement)
    if ticket.get("issue_type") == "where_is_my_order" and logistics.get("status") == "in_transit":
        email_engagement = customer.get("email_engagement")
        if email_engagement in ["high", "medium"]:
            return "friendly_status_check"
        return "simple_status_check"

    # Default fallback
    return "unknown_issue"


def determine_resolution_path(issue_type: str) -> List[str]:
    """
    Determine the resolution path (list of specialist agents) for a given issue type.

    Args:
        issue_type: The classified issue type

    Returns:
        Ordered list of specialist agent IDs to call
    """
    rules = {
        "lost_package": [
            "refund_agent",
            "apology_message_agent"
        ],
        "delivery_delay": [
            "shipping_update_agent",
            "apology_message_agent"
        ],
        "delivery_delay_with_churn_risk": [
            "shipping_update_agent",
            "apology_message_agent",
            "escalation_agent"
        ],
        "warehouse_delay": [
            "shipping_update_agent",
            "apology_message_agent"
        ],
        "warehouse_delay_high_churn_risk": [
            "shipping_update_agent",
            "apology_message_agent",
            "escalation_agent"
        ],
        "item_not_received": [
            "escalation_agent",
            "apology_message_agent"
        ],
        "repeat_item_not_received": [
            "escalation_agent",
            "refund_agent"
        ],
        "simple_status_check": [
            "shipping_update_agent"
        ],
        "friendly_status_check": [
            "shipping_update_agent"
        ],
    }
    return rules.get(issue_type, [])


def determine_expected_outcome(issue_type: str) -> str:
    """
    Determine the expected outcome for a given issue type.

    Args:
        issue_type: The classified issue type

    Returns:
        Expected outcome string
    """
    outcomes = {
        "lost_package": "issue_refund_and_notify_customer",
        "delivery_delay": "acknowledge_delay_and_update_eta",
        "delivery_delay_with_churn_risk": "resolve_delay_and_prevent_churn",
        "warehouse_delay": "explain_warehouse_issue_and_update_eta",
        "warehouse_delay_high_churn_risk": "prevent_churn_and_provide_clear_update",
        "item_not_received": "escalate_and_initiate_investigation",
        "repeat_item_not_received": "immediate_escalation_and_refund",
        "simple_status_check": "provide_delivery_update",
        "friendly_status_check": "provide_eta_update"
    }
    return outcomes.get(issue_type, "unknown_outcome")



# issue classification node

In [ ]:
"""Node 3: Issue Classification - Classify final issue type using decision rules"""

from typing import Dict, Any
from config import CustomerJourneyOrchestratorState, CustomerJourneyOrchestratorConfig
from utils.decision_rules import (
    classify_issue,
    determine_resolution_path,
    determine_expected_outcome
)


def issue_classification_node(
    state: CustomerJourneyOrchestratorState,
    config: CustomerJourneyOrchestratorConfig
) -> Dict[str, Any]:
    """
    Node 3: Issue Classification

    This node:
    1. Takes data from Node 1 (customer, order, logistics)
    2. Takes ticket data from Node 2
    3. Uses decision rules to classify the final issue type
    4. Determines the resolution path (which agents to call)
    5. Determines the expected outcome

    Args:
        state: Current orchestrator state (should have data from Nodes 1 & 2)
        config: Orchestrator configuration

    Returns:
        Updated state with issue_type, resolution_path, and expected_outcome
    """
    errors = state.get("errors", [])

    # Get required data from previous nodes
    customer_data = state.get("customer_data")
    order_data = state.get("order_data")
    logistics_data = state.get("logistics_data")
    ticket_data = state.get("ticket_data")

    # Validate required data
    if not customer_data:
        errors.append("customer_data is required for issue classification (run Node 1 first)")
        return {"errors": errors}

    if not order_data:
        errors.append("order_data is required for issue classification (run Node 1 first)")
        return {"errors": errors}

    if not logistics_data:
        errors.append("logistics_data is required for issue classification (run Node 1 first)")
        return {"errors": errors}

    if not ticket_data:
        errors.append("ticket_data is required for issue classification (run Node 2 first)")
        return {"errors": errors}

    try:
        # Classify the issue using decision rules
        issue_type = classify_issue(
            order=order_data,
            ticket=ticket_data,
            customer=customer_data,
            logistics=logistics_data
        )

        # Determine resolution path (which agents to call)
        resolution_path = determine_resolution_path(issue_type)

        # Determine expected outcome
        expected_outcome = determine_expected_outcome(issue_type)

        # Prepare updated state
        updates: Dict[str, Any] = {
            "issue_type": issue_type,
            "resolution_path": resolution_path,
            "expected_outcome": expected_outcome,
            "errors": errors,  # Always include errors field
        }

        return updates

    except Exception as e:
        errors.append(f"Error during issue classification: {str(e)}")
        return {"errors": errors}



# test issue classifcation

In [ ]:
"""Test Node 3: Issue Classification"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))

from config import CustomerJourneyOrchestratorState, CustomerJourneyOrchestratorConfig
from nodes.data_aggregation_node import data_aggregation_node
from nodes.ticket_creation_node import ticket_creation_node
from nodes.issue_classification_node import issue_classification_node


def test_issue_classification_node():
    """Test that Node 3 correctly classifies issues"""

    print("\n" + "="*60)
    print("TEST 1: Issue Classification - Simple Status Check")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    # Run Nodes 1 & 2 first
    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C001",
        "order_id": "O1001",
        "customer_message": "Hi, my order hasn't arrived yet. The tracking hasn't updated in a while. Can you check?",
        "errors": []
    }

    # Node 1
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    # Node 2
    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    # Node 3
    node3_result = issue_classification_node(state, config)
    state.update(node3_result)

    # Check results
    assert "errors" in node3_result, "Result should contain errors field"

    if node3_result.get("errors"):
        print(f"❌ ERRORS FOUND: {node3_result['errors']}")
        return False

    # Verify classification results
    assert "issue_type" in node3_result, "Should have issue_type"
    assert "resolution_path" in node3_result, "Should have resolution_path"
    assert "expected_outcome" in node3_result, "Should have expected_outcome"

    # For C001 (high engagement, in_transit) -> friendly_status_check
    assert node3_result["issue_type"] == "friendly_status_check", \
        f"Expected friendly_status_check, got {node3_result['issue_type']}"
    assert len(node3_result["resolution_path"]) > 0, "Should have at least one agent in path"

    print("✅ All assertions passed!")
    print(f"\nIssue Type: {node3_result['issue_type']}")
    print(f"Resolution Path: {' → '.join(node3_result['resolution_path'])}")
    print(f"Expected Outcome: {node3_result['expected_outcome']}")

    return True


def test_delivery_delay_classification():
    """Test delivery delay classification with churn risk"""

    print("\n" + "="*60)
    print("TEST 2: Delivery Delay Classification")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    # C002 has high churn risk (0.28) and delayed order
    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C002",
        "order_id": "O1002",
        "customer_message": "My package is delayed again. This is really frustrating. What's going on?",
        "errors": []
    }

    # Run all nodes
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    node3_result = issue_classification_node(state, config)
    state.update(node3_result)

    # C002 has churn_risk 0.28 (> 0.25) and delayed status
    # Should classify as delivery_delay_with_churn_risk
    assert node3_result["issue_type"] == "delivery_delay_with_churn_risk", \
        f"Expected delivery_delay_with_churn_risk, got {node3_result['issue_type']}"

    # Should include escalation_agent in path
    assert "escalation_agent" in node3_result["resolution_path"], \
        "High churn risk should trigger escalation"

    print("✅ Classification correct!")
    print(f"Issue Type: {node3_result['issue_type']}")
    print(f"Resolution Path: {' → '.join(node3_result['resolution_path'])}")
    print(f"Customer Churn Risk: {state['customer_data']['churn_risk']}")
    print(f"Logistics Status: {state['logistics_data']['status']}")

    return True


def test_lost_package_classification():
    """Test lost package classification"""

    print("\n" + "="*60)
    print("TEST 3: Lost Package Classification")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    # C003 has lost package
    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C003",
        "order_id": "O1003",
        "customer_message": "The tracking page says unavailable. Is my package lost?",
        "errors": []
    }

    # Run all nodes
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    node3_result = issue_classification_node(state, config)
    state.update(node3_result)

    # Should classify as lost_package
    assert node3_result["issue_type"] == "lost_package", \
        f"Expected lost_package, got {node3_result['issue_type']}"

    # Should include refund_agent in path
    assert "refund_agent" in node3_result["resolution_path"], \
        "Lost package should trigger refund"

    print("✅ Classification correct!")
    print(f"Issue Type: {node3_result['issue_type']}")
    print(f"Resolution Path: {' → '.join(node3_result['resolution_path'])}")
    print(f"Logistics Status: {state['logistics_data']['status']}")

    return True


def test_warehouse_delay_classification():
    """Test warehouse delay classification with high churn risk"""

    print("\n" + "="*60)
    print("TEST 4: Warehouse Delay with High Churn Risk")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    # C004 has high churn risk (0.35) and warehouse delay
    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C004",
        "order_id": "O1004",
        "customer_message": "My order hasn't left the warehouse yet. Is something wrong with the shipment?",
        "errors": []
    }

    # Run all nodes
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    node3_result = issue_classification_node(state, config)
    state.update(node3_result)

    # C004 has churn_risk 0.35 (> 0.30) and warehouse_issue_flag = True
    # Should classify as warehouse_delay_high_churn_risk
    assert node3_result["issue_type"] == "warehouse_delay_high_churn_risk", \
        f"Expected warehouse_delay_high_churn_risk, got {node3_result['issue_type']}"

    # Should include escalation_agent
    assert "escalation_agent" in node3_result["resolution_path"], \
        "High churn risk should trigger escalation"

    print("✅ Classification correct!")
    print(f"Issue Type: {node3_result['issue_type']}")
    print(f"Resolution Path: {' → '.join(node3_result['resolution_path'])}")
    print(f"Customer Churn Risk: {state['customer_data']['churn_risk']}")
    print(f"Warehouse Issue Flag: {state['order_data']['warehouse_issue_flag']}")

    return True


def test_end_to_end_nodes_1_2_3():
    """Test that all three nodes work together"""

    print("\n" + "="*60)
    print("TEST 5: End-to-End (Node 1 → Node 2 → Node 3)")
    print("="*60)

    config = CustomerJourneyOrchestratorConfig()

    state: CustomerJourneyOrchestratorState = {
        "customer_id": "C001",
        "order_id": "O1001",
        "customer_message": "Hi, my order hasn't arrived yet. The tracking hasn't updated in a while. Can you check?",
        "errors": []
    }

    # Run all three nodes
    print("Running Node 1: Data Aggregation...")
    node1_result = data_aggregation_node(state, config)
    state.update(node1_result)

    print("Running Node 2: Ticket Creation...")
    node2_result = ticket_creation_node(state, config)
    state.update(node2_result)

    print("Running Node 3: Issue Classification...")
    node3_result = issue_classification_node(state, config)
    state.update(node3_result)

    # Verify final state
    assert "customer_data" in state, "Should have customer_data"
    assert "order_data" in state, "Should have order_data"
    assert "logistics_data" in state, "Should have logistics_data"
    assert "ticket_data" in state, "Should have ticket_data"
    assert "issue_type" in state, "Should have issue_type"
    assert "resolution_path" in state, "Should have resolution_path"
    assert "expected_outcome" in state, "Should have expected_outcome"

    print("✅ All nodes work together correctly!")
    print(f"\nFinal State Summary:")
    print(f"  Customer: {state['customer_data']['name']} (churn risk: {state['customer_data']['churn_risk']})")
    print(f"  Order: {state['order_data']['order_id']} (warehouse issue: {state['order_data']['warehouse_issue_flag']})")
    print(f"  Logistics: {state['logistics_data']['status']} ({state['logistics_data']['carrier']})")
    print(f"  Ticket Issue Type (initial): {state['ticket_data']['issue_type']}")
    print(f"  Final Issue Type: {state['issue_type']}")
    print(f"  Resolution Path: {' → '.join(state['resolution_path'])}")
    print(f"  Expected Outcome: {state['expected_outcome']}")

    return True


if __name__ == "__main__":
    print("\n🧪 Testing Node 3: Issue Classification")
    print("="*60)

    results = []

    try:
        results.append(("Test 1: Simple Status Check", test_issue_classification_node()))
    except Exception as e:
        print(f"❌ Test 1 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 1: Simple Status Check", False))

    try:
        results.append(("Test 2: Delivery Delay", test_delivery_delay_classification()))
    except Exception as e:
        print(f"❌ Test 2 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 2: Delivery Delay", False))

    try:
        results.append(("Test 3: Lost Package", test_lost_package_classification()))
    except Exception as e:
        print(f"❌ Test 3 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 3: Lost Package", False))

    try:
        results.append(("Test 4: Warehouse Delay", test_warehouse_delay_classification()))
    except Exception as e:
        print(f"❌ Test 4 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 4: Warehouse Delay", False))

    try:
        results.append(("Test 5: End-to-End", test_end_to_end_nodes_1_2_3()))
    except Exception as e:
        print(f"❌ Test 5 failed with exception: {e}")
        import traceback
        traceback.print_exc()
        results.append(("Test 5: End-to-End", False))

    # Summary
    print("\n" + "="*60)
    print("TEST SUMMARY")
    print("="*60)
    for test_name, passed in results:
        status = "✅ PASSED" if passed else "❌ FAILED"
        print(f"{status}: {test_name}")

    all_passed = all(result[1] for result in results)
    if all_passed:
        print("\n🎉 All tests passed! Node 3 is working correctly.")
    else:
        print("\n⚠️  Some tests failed. Please review the errors above.")

    sys.exit(0 if all_passed else 1)



# test results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_027_Customer_Journey_Orchestrator % cd /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_027_Customer_Journey_Orchestrator && python3 tests/test_node3_issue_classification.py

🧪 Testing Node 3: Issue Classification
============================================================

============================================================
TEST 1: Issue Classification - Simple Status Check
============================================================
✅ All assertions passed!

Issue Type: friendly_status_check
Resolution Path: shipping_update_agent
Expected Outcome: provide_eta_update

============================================================
TEST 2: Delivery Delay Classification
============================================================
✅ Classification correct!
Issue Type: delivery_delay_with_churn_risk
Resolution Path: shipping_update_agent → apology_message_agent → escalation_agent
Customer Churn Risk: 0.28
Logistics Status: delayed

============================================================
TEST 3: Lost Package Classification
============================================================
✅ Classification correct!
Issue Type: lost_package
Resolution Path: refund_agent → apology_message_agent
Logistics Status: lost

============================================================
TEST 4: Warehouse Delay with High Churn Risk
============================================================
✅ Classification correct!
Issue Type: warehouse_delay_high_churn_risk
Resolution Path: shipping_update_agent → apology_message_agent → escalation_agent
Customer Churn Risk: 0.35
Warehouse Issue Flag: True

============================================================
TEST 5: End-to-End (Node 1 → Node 2 → Node 3)
============================================================
Running Node 1: Data Aggregation...
Running Node 2: Ticket Creation...
Running Node 3: Issue Classification...
✅ All nodes work together correctly!

Final State Summary:
  Customer: Sarah Lee (churn risk: 0.12)
  Order: O1001 (warehouse issue: False)
  Logistics: in_transit (FedEx)
  Ticket Issue Type (initial): where_is_my_order
  Final Issue Type: friendly_status_check
  Resolution Path: shipping_update_agent
  Expected Outcome: provide_eta_update

============================================================
TEST SUMMARY
============================================================
✅ PASSED: Test 1: Simple Status Check
✅ PASSED: Test 2: Delivery Delay
✅ PASSED: Test 3: Lost Package
✅ PASSED: Test 4: Warehouse Delay
✅ PASSED: Test 5: End-to-End

🎉 All tests passed! Node 3 is working correctly.


# Decision Rules Management: From Code to Production

**Key Question:** How do businesses create, maintain, and update decision rules in production?

---

## 🎯 The Problem

**Current Approach (What We Have):**
```python
# Hardcoded in Python
if logistics.get("status") == "delayed":
    if churn_risk > 0.25:
        return "delivery_delay_with_churn_risk"
```

**Problems:**
- ❌ Business users can't update rules (need developers)
- ❌ Code changes required for every rule update
- ❌ Risk of bugs when updating code
- ❌ Hard to audit "what changed when"
- ❌ Can't A/B test different rules
- ❌ Slow to respond to business needs

---

## 🏗️ Production Approaches

### Approach 1: Configuration Files (JSON/YAML) ⭐ Most Common

**How It Works:**
```json
// rules.json (stored in database or config service)
{
  "issue_classification": {
    "delivery_delay": {
      "conditions": {
        "logistics.status": ["delayed"],
        "customer.churn_risk": [">0.25"]
      },
      "result": "delivery_delay_with_churn_risk"
    }
  }
}
```

**Code Reads Rules:**
```python
# Load rules from file/database
rules = load_rules_from_config("rules.json")

# Evaluate rules dynamically
issue_type = evaluate_rules(rules, order, ticket, customer, logistics)
```

**Pros:**
- ✅ Business users can update JSON (with validation)
- ✅ No code changes needed
- ✅ Version control (Git)
- ✅ Easy to audit changes
- ✅ Can A/B test different rule sets

**Cons:**
- ⚠️ Still need to code the rule evaluator
- ⚠️ Complex rules can be hard to express in JSON

**Used By:**
- Shopify (rule-based routing)
- Stripe (payment rules)
- Zendesk (ticket routing)

---

### Approach 2: Rule Engine (Drools, OpenL Tablets) ⭐ Enterprise

**How It Works:**
```java
// Business rules in Drools (Java-based rule engine)
rule "Delivery Delay with High Churn Risk"
    when
        logistics.status == "delayed"
        customer.churnRisk > 0.25
    then
        issueType = "delivery_delay_with_churn_risk"
end
```

**Or OpenL Tablets (Excel-based):**
```
| Condition 1        | Condition 2      | Result                    |
|---------------------|------------------|---------------------------|
| logistics.status    | churn_risk       | issue_type                |
| "delayed"          | > 0.25          | delivery_delay_with_churn |
```

**Pros:**
- ✅ Business-friendly (Excel-like interface)
- ✅ Powerful rule engine
- ✅ Version control
- ✅ Audit trail
- ✅ Used by Fortune 500 companies

**Cons:**
- ⚠️ Requires rule engine infrastructure
- ⚠️ Learning curve
- ⚠️ Can be overkill for simple rules

**Used By:**
- Banks (fraud detection)
- Insurance companies (claims processing)
- Large enterprises (compliance rules)

---

### Approach 3: Database-Driven Rules ⭐ Flexible

**How It Works:**
```sql
-- Rules stored in database
CREATE TABLE decision_rules (
    rule_id INT PRIMARY KEY,
    rule_name VARCHAR(255),
    conditions JSON,
    result VARCHAR(255),
    priority INT,
    active BOOLEAN,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
);

-- Example rule
INSERT INTO decision_rules VALUES (
    1,
    'Delivery Delay with High Churn',
    '{"logistics.status": "delayed", "customer.churn_risk": ">0.25"}',
    'delivery_delay_with_churn_risk',
    10,
    true,
    NOW(),
    NOW()
);
```

**Code Evaluates Rules:**
```python
# Load active rules from database
rules = db.query("SELECT * FROM decision_rules WHERE active = true ORDER BY priority")

# Evaluate each rule
for rule in rules:
    if evaluate_conditions(rule.conditions, state):
        return rule.result
```

**Pros:**
- ✅ Business users can update via UI
- ✅ Real-time updates (no deployment)
- ✅ Version history in database
- ✅ Can enable/disable rules instantly
- ✅ A/B testing (multiple active rules)

**Cons:**
- ⚠️ Need to build rule management UI
- ⚠️ Database queries add latency
- ⚠️ Need caching for performance

**Used By:**
- E-commerce platforms (dynamic pricing)
- Marketing platforms (campaign rules)
- SaaS platforms (feature flags)

---

### Approach 4: LLM-Based Rule Interpretation ⭐ Emerging

**How It Works:**
```python
# Rules stored as natural language in document
rules_doc = """
Rule 1: If logistics status is "delayed" AND customer churn risk > 0.25,
        classify as "delivery_delay_with_churn_risk"

Rule 2: If warehouse_issue_flag is True AND churn risk > 0.30,
        classify as "warehouse_delay_high_churn_risk"
...
"""

# LLM interprets rules
prompt = f"""
Given these business rules:
{rules_doc}

And this customer situation:
- Logistics status: {logistics.status}
- Churn risk: {customer.churn_risk}
- Warehouse issue: {order.warehouse_issue_flag}

Classify the issue type according to the rules.
"""

issue_type = llm.classify(prompt)
```

**Pros:**
- ✅ Natural language rules (business-friendly)
- ✅ No code changes needed
- ✅ Can handle complex, ambiguous rules
- ✅ Easy to update (just edit document)

**Cons:**
- ⚠️ LLM can be inconsistent
- ⚠️ Hard to debug "why did it choose this?"
- ⚠️ Latency (LLM calls are slow)
- ⚠️ Cost (API calls)
- ⚠️ Less deterministic than code

**Used By:**
- Early-stage startups (rapid iteration)
- Research projects
- Not yet mainstream in production

---

## 🎯 Recommended Approach: Hybrid

**Best Practice:** Use different approaches for different rule types

### Simple Rules → Configuration Files
```json
// Simple, deterministic rules
{
  "if logistics.status == 'lost'": "lost_package",
  "if churn_risk > 0.25": "high_priority"
}
```

### Complex Rules → Rule Engine
```java
// Complex business logic
rule "Multi-factor Churn Risk"
    when
        customer.churnRisk > 0.25
        AND customer.engagement == "low"
        AND order.delayDays > 5
    then
        issueType = "critical_churn_risk"
        escalate()
end
```

### Ambiguous Rules → LLM
```python
# Rules that need interpretation
rules = """
If customer seems frustrated based on message tone,
and has high value, escalate immediately.
"""
```

---

## 📊 Real-World Examples

### Example 1: Shopify (E-commerce)

**Approach:** Configuration Files + Database

```yaml
# routing_rules.yaml
fulfillment_rules:
  - condition: order.value > 1000
    route_to: "priority_fulfillment"
  - condition: customer.loyalty_tier == "gold"
    route_to: "express_fulfillment"
```

**Management:**
- Rules stored in Git
- Business team updates YAML files
- Code automatically reloads rules
- Version control for audit

---

### Example 2: Stripe (Payments)

**Approach:** Database-Driven Rules

```sql
-- Fraud detection rules
SELECT * FROM fraud_rules
WHERE active = true
ORDER BY priority;

-- Rules can be updated via admin UI
-- Changes take effect immediately
```

**Management:**
- Admin UI for rule management
- A/B testing framework
- Real-time rule updates
- Audit log of all changes

---

### Example 3: Zendesk (Support)

**Approach:** Rule Engine (Drools)

```java
// Ticket routing rules
rule "High-Value Customer Escalation"
    when
        ticket.customer.tier == "enterprise"
        ticket.priority == "high"
    then
        routeTo("tier_2_support")
        notify("account_manager")
end
```

**Management:**
- Business analysts update rules
- Excel-like interface (OpenL Tablets)
- Version control
- Testing environment before production

---

## 🔧 Implementation Patterns

### Pattern 1: Rule Versioning

```python
# Store multiple versions of rules
rules_v1 = load_rules("rules_v1.json")
rules_v2 = load_rules("rules_v2.json")

# A/B test
if customer_id % 2 == 0:
    rules = rules_v1
else:
    rules = rules_v2
```

### Pattern 2: Rule Validation

```python
# Validate rules before applying
def validate_rule(rule):
    # Check syntax
    # Check for conflicts
    # Check for missing conditions
    pass

# Only apply validated rules
if validate_rule(new_rule):
    apply_rule(new_rule)
```

### Pattern 3: Rule Testing

```python
# Test rules with known scenarios
test_cases = [
    {
        "input": {"status": "delayed", "churn_risk": 0.30},
        "expected": "delivery_delay_with_churn_risk"
    }
]

# Run tests before deploying rules
for test in test_cases:
    result = evaluate_rules(rules, test["input"])
    assert result == test["expected"]
```

---

## 💡 LLM for Rule Interpretation: Pros & Cons

### When LLM Makes Sense

**✅ Good For:**
- Ambiguous rules ("if customer seems frustrated")
- Rapid prototyping (no code needed)
- Natural language rules (business-friendly)
- Complex pattern matching

**Example:**
```python
# Business rule in natural language
rule = "If customer message indicates high frustration
        AND they're a gold tier customer, escalate immediately"

# LLM interprets
if llm.evaluate_rule(rule, customer_message, customer_tier):
    escalate()
```

### When LLM Doesn't Make Sense

**❌ Bad For:**
- Deterministic rules (need consistency)
- High-volume decisions (latency/cost)
- Regulatory compliance (need audit trail)
- Simple if/then logic

**Example:**
```python
# Simple rule - use code, not LLM
if churn_risk > 0.25:
    escalate()
# LLM would be overkill and inconsistent
```

---

## 🎯 Recommended Architecture

### For Your Orchestrator (Production-Ready)

```
┌─────────────────────────────────────┐
│   Rule Management UI                │
│   (Business users update rules)     │
└──────────────┬──────────────────────┘
               │
               ↓
┌─────────────────────────────────────┐
│   Rules Database                     │
│   - Active rules                     │
│   - Version history                  │
│   - A/B test variants                │
└──────────────┬──────────────────────┘
               │
               ↓
┌─────────────────────────────────────┐
│   Rule Cache (Redis)                 │
│   - Fast access                     │
│   - Auto-refresh on updates         │
└──────────────┬──────────────────────┘
               │
               ↓
┌─────────────────────────────────────┐
│   Rule Evaluator (Your Code)        │
│   - Loads rules from cache          │
│   - Evaluates conditions            │
│   - Returns classification          │
└──────────────────────────────────────┘
```

**Benefits:**
- ✅ Business users can update rules
- ✅ Fast (cached rules)
- ✅ Version control
- ✅ A/B testing
- ✅ Audit trail

---

## 📝 Rule Maintenance Best Practices

### 1. **Version Control**
```git
# Rules in Git
rules/
  ├── v1/
  │   └── classification_rules.json
  ├── v2/
  │   └── classification_rules.json
  └── current -> v2/
```

### 2. **Testing**
```python
# Test rules before deploying
def test_rules():
    test_scenarios = load_test_scenarios()
    for scenario in test_scenarios:
        result = evaluate_rules(scenario)
        assert result == scenario.expected
```

### 3. **Monitoring**
```python
# Track rule performance
def log_rule_decision(rule_id, input_data, result):
    logger.info({
        "rule_id": rule_id,
        "input": input_data,
        "result": result,
        "timestamp": now()
    })
```

### 4. **Gradual Rollout**
```python
# Roll out rules gradually
if customer_id % 100 < 10:  # 10% rollout
    use_new_rules()
else:
    use_old_rules()
```

---

## 🚀 Migration Path

### Phase 1: Current (Hardcoded)
```python
# What we have now
if logistics.get("status") == "delayed":
    return "delivery_delay"
```

### Phase 2: Configuration Files
```python
# Load from JSON
rules = load_rules("rules.json")
result = evaluate_rules(rules, state)
```

### Phase 3: Database-Driven
```python
# Load from database
rules = db.get_active_rules()
result = evaluate_rules(rules, state)
```

### Phase 4: Hybrid (LLM for Complex)
```python
# Simple rules: code
# Complex rules: LLM
if rule.complexity == "simple":
    result = evaluate_code_rule(rule)
else:
    result = llm.evaluate_rule(rule)
```

---

## ✅ Summary

### For Your Project (Now)
- ✅ Keep rules in code (simple, works)
- ✅ Document rules clearly
- ✅ Test thoroughly

### For Production (Later)
- ✅ Move to configuration files (JSON/YAML)
- ✅ Add rule management UI
- ✅ Version control rules
- ✅ A/B test rule changes

### For Enterprise (Future)
- ✅ Database-driven rules
- ✅ Rule engine (Drools/OpenL)
- ✅ Business user interface
- ✅ Full audit trail

### LLM for Rules?
- ✅ Use for ambiguous/complex rules
- ❌ Don't use for simple if/then logic
- ⚠️ Consider latency and cost
- ⚠️ Need fallback to deterministic rules

**The key insight:** Start simple (code), evolve to config files, then database/rule engine as you scale. LLM is a tool for specific use cases, not a replacement for structured rules.

